In [50]:
import os

def walk_through_dir(dir_path):
    """
    Walks through dir_path returning its contents.
    Args:
        dir_path (str or pathlib.Path): target directory

    Returns:
        A print out of:
            number of subdirectories in dir_path
            number of images (files) in each subdirectory
            name of each subdirectory
    """
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")


image_path = r"C:\Users\ashmi\Downloads\pizza_steak_sushi_20_percent"


print("Image path:", image_path)


walk_through_dir(image_path)

from pathlib import Path


image_path = Path(r"C:\Users\ashmi\Downloads\pizza_steak_sushi_20_percent")

train_dir = image_path / "train"
test_dir = image_path / "test"

print("train data -", train_dir, "  test data -", test_dir)


Image path: C:\Users\ashmi\Downloads\pizza_steak_sushi_20_percent
There are 2 directories and 0 images in 'C:\Users\ashmi\Downloads\pizza_steak_sushi_20_percent'.
There are 3 directories and 0 images in 'C:\Users\ashmi\Downloads\pizza_steak_sushi_20_percent\test'.
There are 0 directories and 46 images in 'C:\Users\ashmi\Downloads\pizza_steak_sushi_20_percent\test\pizza'.
There are 0 directories and 58 images in 'C:\Users\ashmi\Downloads\pizza_steak_sushi_20_percent\test\steak'.
There are 0 directories and 46 images in 'C:\Users\ashmi\Downloads\pizza_steak_sushi_20_percent\test\sushi'.
There are 3 directories and 0 images in 'C:\Users\ashmi\Downloads\pizza_steak_sushi_20_percent\train'.
There are 0 directories and 154 images in 'C:\Users\ashmi\Downloads\pizza_steak_sushi_20_percent\train\pizza'.
There are 0 directories and 146 images in 'C:\Users\ashmi\Downloads\pizza_steak_sushi_20_percent\train\steak'.
There are 0 directories and 150 images in 'C:\Users\ashmi\Downloads\pizza_steak_sus

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transforms = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

C:\Users\ashmi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_simple = datasets.ImageFolder(root=train_dir, transform=train_transforms)
test_data_simple = datasets.ImageFolder(root=test_dir, transform=test_transforms)


BATCH_SIZE = 32
NUM_WORKERS = 16

train_dataloader_simple = DataLoader(dataset=train_data_simple, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dataloader_simple = DataLoader(dataset=test_data_simple, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

NUM_WORKERS


16

In [53]:
import torch
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class AlexNet(nn.Module):
    """
    Accurate implementation of the AlexNet architecture
    """
    def __init__(self, input_shape: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=96, 
                      kernel_size=11, 
                      stride=4, 
                      padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=96, 
                      out_channels=256, 
                      kernel_size=5, 
                      stride=1, 
                      padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)  # Corrected kernel size
        )
        self.conv_block_3 = nn.Sequential(
            nn.Conv2d(in_channels=256, 
                      out_channels=384, 
                      kernel_size=3, 
                      stride=1, 
                      padding=1),
            nn.ReLU(inplace=True)
        )
        self.conv_block_4 = nn.Sequential(
            nn.Conv2d(in_channels=384, 
                      out_channels=384, 
                      kernel_size=3, 
                      stride=1, 
                      padding=1),
            nn.ReLU(inplace=True)
        )
        self.conv_block_5 = nn.Sequential(
            nn.Conv2d(in_channels=384, 
                      out_channels=256, 
                      kernel_size=3, 
                      stride=1, 
                      padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, output_shape),
        )

    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.conv_block_3(x)
        x = self.conv_block_4(x)
        x = self.conv_block_5(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_001 = AlexNet(input_shape=3, output_shape=len(train_data_simple.classes)).to(device)

In [54]:
def accuracy_fn(y_true, y_pred):
    _, predicted = torch.max(y_pred, 1)
    correct = (predicted == y_true).sum().item()
    accuracy = correct / y_true.shape[0]
    return accuracy


loss_fn = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(params=model_001.parameters(), lr=0.00001)

In [55]:
from timeit import default_timer as timer 
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format). 
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [56]:
import torch
from tqdm.auto import tqdm
from timeit import default_timer as timer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_001.to(device)


torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 10


for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n-------")

    train_loss = 0

    for batch, (X, y) in enumerate(train_dataloader_simple):
        model_001.train() 
   
        X, y = X.to(device), y.to(device)
        
       
        y_pred = model_001(X)

        
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

      
        optimizer.zero_grad()

        
        loss.backward()

        optimizer.step()

        
        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(train_dataloader_simple.dataset)} samples")


    train_loss /= len(train_dataloader_simple)
    

    test_loss, test_acc = 0, 0 
    model_001.eval()
    with torch.inference_mode():
        for X, y in test_dataloader_simple:
       
            X, y = X.to(device), y.to(device)
            
      
            test_pred = model_001(X)
      
            test_loss += loss_fn(test_pred, y).item() 
       
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1).unsqueeze(-1))
        
        
        test_loss /= len(test_dataloader_simple)

   
        test_acc /= len(test_dataloader_simple)


    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f}, Test acc: {test_acc * 100:.2f}%\n")

   
train_time_end_on_cpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_cpu, 
                                           end=train_time_end_on_cpu,
                                           device=str(next(model_001.parameters()).device))


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
-------
Looked at 0/450 samples


 10%|█         | 1/10 [00:23<03:29, 23.29s/it]


Train loss: 1.09823 | Test loss: 1.09764, Test acc: 30.74%

Epoch: 1
-------
Looked at 0/450 samples


 20%|██        | 2/10 [00:47<03:09, 23.73s/it]

...


Train loss: 1.09994 | Test loss: 1.09755, Test acc: 31.31%

Epoch: 8
-------
Looked at 0/450 samples


 90%|█████████ | 9/10 [03:32<00:23, 23.38s/it]


Train loss: 1.09903 | Test loss: 1.09776, Test acc: 30.45%

Epoch: 9
-------
Looked at 0/450 samples


100%|██████████| 10/10 [03:56<00:00, 23.65s/it]


Train loss: 1.09903 | Test loss: 1.09750, Test acc: 31.02%

Train time on cuda:0: 236.542 seconds


In [57]:
torch.save(model_001.state_dict(), "model_011.pth")